In [5]:
from typing import TypedDict,List,Union
from langchain_core.messages import HumanMessage,AIMessage
from langgraph.graph import START,END,StateGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pathlib import Path
import os

In [4]:
dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)
api_key=os.getenv("gptAPIKEY")
# print(api_key)

In [6]:
class Agent(TypedDict):
    message : List[Union[HumanMessage,AIMessage]]

In [7]:
llm= ChatOpenAI(model="openai/gpt-oss-20b:free",api_key=api_key,base_url="https://openrouter.ai/api/v1")

In [11]:
def process(state:Agent)->Agent:
    "calls llm"
    response=llm.invoke(state['message'])
    print("AI ",response.content)
    state['message'].append(AIMessage(content=response.content))
    return state

In [12]:
graph=StateGraph(Agent)
graph.add_node("process",process)
graph.add_edge(START,"process")
graph.add_edge("process",END)
agent=graph.compile()

In [ ]:
conversationhistory=[]
userinput= input("Enter: ")

while userinput !="exit":
    conversationhistory.append(HumanMessage(content=userinput))
    result= agent.invoke({"message":conversationhistory})
    conversationhistory=result["message"]
    userinput= input("Enter: ")

with open("log.txt","w") as file:
    for i in conversationhistory:
        if isinstance(i,HumanMessage):
            file.write("you ",i)
        elif isinstance(i,AIMessage):
            file.write("ai ",i)

AI  Hello! 👋 How can I help you today?
AI  Nice to meet you, Hero! 🌟 How can I help you today?
AI  You’ve introduced yourself as **Hero**. If you’d like to add anything else or change it, just let me know!
